In [0]:
import pyspark
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

# Filling config params with dummy data throughout
spark.conf.set("mycontainer@mystorage","somekey")

# Read Trade Partition Dataset From It’s Temporary Location
trade_common = spark.read.format('parquet').load("wasbs://mycontainer@mystorage.blob.core.windows.net/output_dir/partition=T")

# Select The Necessary Columns For Trade Records
trade = trade_common.select("trade_dt", "rec_type", "symbol", "exchange", "event_tm", "event_seq_nb", "arrival_tm", "trade_pr")

# Apply Data Correction
def applyLatest(df):
    #trades
    if df.first()["rec_type"] == "T":
        df_grouped = df.groupBy("trade_dt", "rec_type", "symbol", "arrival_tm", "event_seq_nb").agg(max("event_tm").alias("latest_trade"))
        df_joined = df_grouped.join(df.select("event_tm", "exchange", "trade_pr"), df.event_tm == df_grouped.latest_trade, "inner")
        df_final = df_joined.select("trade_dt", "rec_type", col("symbol").alias("stock_symbol"), col("exchange").alias("stock_exchange"), "latest_trade", "event_seq_nb", "arrival_tm", "trade_pr").orderBy("trade_dt", "symbol", "event_seq_nb")
        return df_final
    #quotes
    elif df.first()["rec_type"] == "Q":
        df_grouped = df.groupBy("trade_dt", "rec_type", "symbol", "arrival_tm", "event_seq_nb").agg(max("event_tm").alias("latest_quote"))
        df_joined = df_grouped.join(df.select("event_tm", "exchange", "bid_pr", "bid_size", "ask_pr", "ask_size"), df.event_tm == df_grouped.latest_quote, "inner")
        df_final = df_joined.select("trade_dt", "rec_type", col("symbol").alias("stock_symbol"), col("exchange").alias("stock_exchange"), "latest_quote", "event_seq_nb", "arrival_tm", "bid_pr", "bid_size", "ask_pr", "ask_size").orderBy("trade_dt", "symbol", "event_seq_nb")
        return df_final

trade_corrected = applyLatest(trade)
trades_080520 = trade_corrected.where(trade_corrected.trade_dt == "2020-08-05")
trades_080620 = trade_corrected.where(trade_corrected.trade_dt == "2020-08-06")

#  Write The Trade Dataset Back To Parquet On Azure Blob Storage
trades_080520.write.parquet("wasbs://somecontainer@somestorage.blob.core.windows.net/trade/trade_dt={}".format('2020-08-05'))
trades_080620.write.parquet("wasbs://somecontainer@somestorage.blob.core.windows.net/trade/trade_dt={}".format('2020-08-06'))

# ****************** REPEAT FOR QUOTES ******************
quote_common = spark.read.format('parquet').load("wasbs://mycontainer@mystorage.blob.core.windows.net/output_dir/partition=Q")

# Select The Necessary Columns For Quote Records
quote = quote_common.select("trade_dt", "rec_type", "symbol", "exchange", "event_tm", "event_seq_nb", "arrival_tm", "bid_pr", "bid_size", "ask_pr", "ask_size")

# Apply Data Correction
quote_corrected = applyLatest(quote)

# Separate dataframes by trade date
quotes_080520 = quote_corrected.where(quote_corrected.trade_dt == "2020-08-05")
quotes_080620 = quote_corrected.where(quote_corrected.trade_dt == "2020-08-06")

# Write The Quote Dataset Back To Parquet On Azure Blob Storage
quotes_080520.write.parquet("wasbs://mycontainer@mystorage.blob.core.windows.net/quote/trade_dt={}".format('2020-08-05'))
quotes_080620.write.parquet("wasbs://mycontainer@mystorage.blob.core.windows.net/quote/trade_dt={}".format('2020-08-06'))




--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-3983592940502891> in <module> 
 10 
 11 # Read Trade Partition Dataset From It’s Temporary Location 
 ---> 12 trade_common = spark . read . format ( 'parquet' ) . load ( "wasbs://mycontainer@mystorage.blob.core.windows.net/output_dir/partition=T" ) 
 13 
 14 # Select The Necessary Columns For Trade Records 

 /databricks/spark/python/pyspark/sql/readwriter.py in load (self, path, format, schema, **options) 
 156 self . options ( ** options ) 
 157 if isinstance ( path , str ) : 
 --> 158 return self . _df ( self . _jreader . load ( path ) ) 
 159 elif path is not None : 
 160 if type ( path ) != list : 

 /databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 115 def deco ( * a , ** kw ) : 
 116 try : 
 --> 117 return f ( * a , ** kw ) 
 118 except py4j . protocol . Py4JJavaError as e : 
 119 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 324 value = OUTPUT_CONVERTER [ type ] ( answer [ 2 : ] , gateway_client ) 
 325 if answer [ 1 ] == REFERENCE_TYPE : 
 --> 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 328 format(target_id, ".", name), value)

 Py4JJavaError : An error occurred while calling o396.load.
: shaded.databricks.org.apache.hadoop.fs.azure.AzureException: shaded.databricks.org.apache.hadoop.fs.azure.AzureException: Container mycontainer in account mystorage.blob.core.windows.net not found, and we can't create it using anoynomous credentials, and no credentials found for them in the configuration.
	at shaded.databricks.org.apache.hadoop.fs.azure.AzureNativeFileSystemStore.createAzureStorageSession(AzureNativeFileSystemStore.java:1063)
	at shaded.databricks.org.apache.hadoop.fs.azure.AzureNativeFileSystemStore.initialize(AzureNativeFileSystemStore.java:512)
	at shaded.databricks.org.apache.hadoop.fs.azure.NativeAzureFileSystem.initialize(NativeAzureFileSystem.java:1383)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3469)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:537)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:55)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:405)
	at org.apache.spark.sql.DataFrameReader.loadV1Source(DataFrameReader.scala:356)
	at org.apache.spark.sql.DataFrameReader.$anonfun$load$2(DataFrameReader.scala:323)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:323)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:236)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)
Caused by: shaded.databricks.org.apache.hadoop.fs.azure.AzureException: Container mycontainer in account mystorage.blob.core.windows.net n